In [ ]:
import os
from sentence_transformers import SentenceTransformer

In [ ]:
import dotenv
dotenv.load_dotenv()

In [ ]:
model = SentenceTransformer('BAAI/bge-small-en-v1.5')

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
MONGODB_URL = os.getenv("MONGODB_URL")

In [ ]:
# Create a new client and connect to the server
mongo_client = MongoClient(MONGODB_URL, server_api=ServerApi('1'))

In [ ]:
question = "What are steps to take when finding projects to build your experience?"

In [ ]:
question_embedding = model.encode(question, normalize_embeddings=True)

In [ ]:
sum(question_embedding)

```json
{
  "fields": [
    {
      "numDimensions": 384,
      "path": "embedding",
      "similarity": "dotProduct",
      "type": "vector"
    },
    {
      "path": "fileName",
      "type": "filter"
    }
  ]
}
```

In [ ]:
query_vector = question_embedding.tolist()

In [ ]:
results = mongo_client["RAG"]['DocumentEmbedding'].aggregate([
    {

        "$vectorSearch": {
            "index": "vector_index",
            "path": "embedding",
            "queryVector": query_vector,
            "numCandidates": 5,
            "limit": 5
        }

    },
    {

        '$project': {
            'embedding': 0,
            "_id": 0,
        }

    }

])

In [ ]:
results = list(results)

In [ ]:
results